# 🛒 UK Retail Purchase Prediction — MLOps Level 0–2

**Dataset:** Online Retail II (UK only)  
**Model:** XGBoost Binary Classifier  
**Goal:** Predict if a customer will purchase next month  

---
### Pipeline
```
Raw CSV → Preprocessing → Feature Engineering → Target Creation
       → Train/Test Split → XGBoost Training → Evaluation
       → Model Saving → Drift Monitoring
```

## 1️⃣ Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay
)
from xgboost import XGBClassifier, plot_importance

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('✅ Libraries loaded')
print(f'   pandas  {pd.__version__}')
print(f'   numpy   {np.__version__}')

## 2️⃣ Load Raw Data

In [ ]:
# ⚠️ Update path to your downloaded Online Retail II CSV
RAW_PATH = 'data/raw.csv'

df = pd.read_csv(RAW_PATH, encoding='ISO-8859-1')
print(f'Shape: {df.shape}')
df.head()

In [ ]:
print('=== Dataset Info ===')
df.info()
print('\n=== Null counts ===')
print(df.isnull().sum())
print('\n=== Countries ===')
print(df['Country'].value_counts().head(10))

## 3️⃣ Data Cleaning & Filtering

In [ ]:
# Filter to UK only
df_uk = df[df['Country'] == 'United Kingdom'].copy()
print(f'UK records: {len(df_uk):,}')

# Drop nulls
df_uk.dropna(subset=['Customer ID'], inplace=True)

# Remove cancellations and invalid rows
df_uk = df_uk[df_uk['Quantity'] > 0]
df_uk = df_uk[df_uk['Price'] > 0]

print(f'After cleaning: {df_uk.shape}')
df_uk.describe()

## 4️⃣ Exploratory Data Analysis (EDA)

In [ ]:
df_uk['InvoiceDate'] = pd.to_datetime(df_uk['InvoiceDate'])
df_uk['Month'] = df_uk['InvoiceDate'].dt.to_period('M').astype(str)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Monthly transaction volume
monthly = df_uk.groupby('Month').size()
monthly.plot(ax=axes[0], kind='bar', color='steelblue')
axes[0].set_title('Monthly Transaction Volume (UK)')
axes[0].set_ylabel('Transactions')
axes[0].tick_params(axis='x', rotation=45)

# Price distribution
df_uk['Price'].clip(0, 50).plot(ax=axes[1], kind='hist', bins=50, color='coral')
axes[1].set_title('Unit Price Distribution (clipped at £50)')
axes[1].set_xlabel('Price (£)')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Hour of purchase
df_uk['InvoiceDate'].dt.hour.value_counts().sort_index().plot(
    ax=axes[0], kind='bar', color='mediumpurple')
axes[0].set_title('Purchases by Hour of Day')
axes[0].set_xlabel('Hour')

# Top customers
top_customers = df_uk.groupby('Customer ID')['Price'].sum().nlargest(10)
top_customers.plot(ax=axes[1], kind='barh', color='teal')
axes[1].set_title('Top 10 Customers by Revenue')
axes[1].set_xlabel('Total Revenue (£)')

plt.tight_layout()
plt.show()

## 5️⃣ Feature Engineering & Target Creation

In [ ]:
df_uk['TotalPrice']    = df_uk['Quantity'] * df_uk['Price']
df_uk['InvoiceHour']   = df_uk['InvoiceDate'].dt.hour
df_uk['InvoiceDay']    = df_uk['InvoiceDate'].dt.day
df_uk['InvoiceMonth']  = df_uk['InvoiceDate'].dt.to_period('M')

print('New features added: TotalPrice, InvoiceHour, InvoiceDay, InvoiceMonth')
df_uk[['Quantity','Price','TotalPrice','InvoiceHour','InvoiceDay']].describe()

In [ ]:
# Aggregate per customer per month
customer_months = (
    df_uk.groupby(['Customer ID', 'InvoiceMonth'])
    .agg(
        Quantity   = ('Quantity',    'sum'),
        UnitPrice  = ('Price',       'mean'),
        TotalPrice = ('TotalPrice',  'sum'),
        InvoiceHour= ('InvoiceHour', 'mean'),
        InvoiceDay = ('InvoiceDay',  'mean'),
    ).reset_index()
)
customer_months['InvoiceMonth'] = customer_months['InvoiceMonth'].astype(str)
customer_months.sort_values(['Customer ID', 'InvoiceMonth'], inplace=True)

# Build lookup set of active (customer, month) pairs
active_set = set(zip(customer_months['Customer ID'], customer_months['InvoiceMonth']))

def next_month_str(period_str):
    return str(pd.Period(period_str, freq='M') + 1)

customer_months['NextMonth'] = customer_months['InvoiceMonth'].apply(next_month_str)

customer_months['WillBuyNextMonth'] = customer_months.apply(
    lambda r: 1 if (r['Customer ID'], r['NextMonth']) in active_set else 0, axis=1
)

print('Target distribution:')
print(customer_months['WillBuyNextMonth'].value_counts())
print(f'\nClass balance: {customer_months["WillBuyNextMonth"].mean():.1%} positive')
customer_months.head()

## 6️⃣ Train / Test Split

In [ ]:
FEATURE_COLS = ['Quantity', 'UnitPrice', 'TotalPrice', 'InvoiceHour', 'InvoiceDay']
TARGET_COL   = 'WillBuyNextMonth'

model_df = customer_months[FEATURE_COLS + [TARGET_COL]].dropna()

X = model_df[FEATURE_COLS]
y = model_df[TARGET_COL]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
)

print(f'Train: {X_train.shape}, Test: {X_test.shape}')

# Save splits
os.makedirs('data', exist_ok=True)
train_df = X_train.copy(); train_df[TARGET_COL] = y_train.values
test_df  = X_test.copy();  test_df[TARGET_COL]  = y_test.values
train_df.to_csv('data/train.csv', index=False)
test_df.to_csv('data/test.csv',   index=False)
print('Saved train.csv and test.csv ✅')

## 7️⃣ Model Training — XGBoost

In [ ]:
model = XGBClassifier(
    n_estimators     = 100,
    max_depth        = 5,
    learning_rate    = 0.1,
    random_state     = RANDOM_STATE,
    eval_metric      = 'logloss',
    use_label_encoder= False,
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=10
)

print('Training complete ✅')

## 8️⃣ Model Evaluation

In [ ]:
y_pred = model.predict(X_test)

metrics = {
    'Accuracy' : accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, zero_division=0),
    'Recall'   : recall_score(y_test, y_pred, zero_division=0),
    'F1-Score' : f1_score(y_test, y_pred, zero_division=0),
}

metrics_df = pd.DataFrame([metrics])
print('=== Evaluation Metrics ===')
display(metrics_df.style.format('{:.4f}'))

print('\n=== Classification Report ===')
print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=['Not Buy', 'Will Buy'])
disp.plot(ax=axes[0], colorbar=False, cmap='Blues')
axes[0].set_title('Confusion Matrix')

# Feature importance
importances = model.feature_importances_
feat_imp = pd.Series(importances, index=FEATURE_COLS).sort_values()
feat_imp.plot(kind='barh', ax=axes[1], color='steelblue')
axes[1].set_title('XGBoost Feature Importance')
axes[1].set_xlabel('Importance Score')

plt.tight_layout()
plt.show()

In [ ]:
# Cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='f1')
print(f'5-Fold Cross Validation F1 Scores: {cv_scores.round(4)}')
print(f'Mean F1: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}')

## 9️⃣ Save Model

In [ ]:
os.makedirs('models', exist_ok=True)
MODEL_PATH = 'models/xgb_model.pkl'
joblib.dump(model, MODEL_PATH)
print(f'Model saved → {MODEL_PATH} ✅')

# Verify reload
loaded_model = joblib.load(MODEL_PATH)
test_pred    = loaded_model.predict(X_test[:3])
print(f'Reload check — sample predictions: {test_pred}')

## 🔟 Drift Monitoring Simulation (MLOps Level 2)

In [ ]:
# ── Baseline predictions ────────────────────────────────────────────────────
original_preds = model.predict(X_test)

# ── Simulate drift: Quantity ×1.2, UnitPrice ×0.8 ──────────────────────────
X_drifted = X_test.copy()
X_drifted['Quantity']   = X_drifted['Quantity']  * 1.2
X_drifted['UnitPrice']  = X_drifted['UnitPrice'] * 0.8
X_drifted['TotalPrice'] = X_drifted['Quantity']  * X_drifted['UnitPrice']

drifted_preds = model.predict(X_drifted)

# ── Compute prediction change ───────────────────────────────────────────────
n_changed   = np.sum(original_preds != drifted_preds)
pct_changed = (n_changed / len(original_preds)) * 100

print(f'Total samples           : {len(original_preds):,}')
print(f'Predictions changed     : {n_changed:,}')
print(f'Prediction change %     : {pct_changed:.2f}%')
print()

DRIFT_THRESHOLD = 10.0
if pct_changed > DRIFT_THRESHOLD:
    print(f'⚠️  DRIFT DETECTED  ({pct_changed:.2f}% > {DRIFT_THRESHOLD}% threshold)')
    print('    → Recommend retraining with fresh data')
else:
    print(f'✅  No significant drift ({pct_changed:.2f}% ≤ {DRIFT_THRESHOLD}% threshold)')

In [ ]:
# Visualise distribution shift
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
features_to_plot = ['Quantity', 'UnitPrice', 'TotalPrice']

for ax, col in zip(axes, features_to_plot):
    ax.hist(X_test[col], bins=40, alpha=0.6, label='Original', color='steelblue')
    ax.hist(X_drifted[col], bins=40, alpha=0.6, label='Drifted',  color='coral')
    ax.set_title(f'{col} Distribution')
    ax.set_xlabel(col)
    ax.legend()

plt.suptitle('Feature Distribution: Original vs Drifted', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Summary table
summary = pd.DataFrame({
    'Feature'         : FEATURE_COLS,
    'Original Mean'   : X_test[FEATURE_COLS].mean().values.round(4),
    'Drifted Mean'    : X_drifted[FEATURE_COLS].mean().values.round(4),
})
summary['Δ%'] = (
    abs(summary['Drifted Mean'] - summary['Original Mean'])
    / (summary['Original Mean'] + 1e-9) * 100
).round(2)

print('=== Feature Drift Summary ===')
display(summary)

print(f'\n=== Final Drift Report ===')
print(f'Prediction Change : {pct_changed:.2f}%')
print('Status            : ' + ('⚠️ DRIFT DETECTED' if pct_changed > DRIFT_THRESHOLD else '✅ OK'))

## ✅ Project Summary

| Component         | Status | Details |
|-------------------|--------|---------|
| Data Preprocessing| ✅     | UK filter, null drops, feature engineering |
| Feature Engineering| ✅    | TotalPrice, InvoiceHour, InvoiceDay |
| Target Creation   | ✅     | Next-month purchase flag |
| Model Training    | ✅     | XGBoost (100 trees, depth 5) |
| Evaluation        | ✅     | Accuracy, Precision, Recall, F1 |
| Model Persistence | ✅     | xgb_model.pkl via joblib |
| Streamlit UI      | ✅     | Single + batch prediction |
| Drift Monitoring  | ✅     | Simulated drift, threshold alert |
| MLOps Level       | ✅     | Level 0 → 2 coverage |

```
streamlit run app.py
```